In [ ]:
import pandas as pd
import numpy as np
from itertools import product
# might be add in requirements.txt
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
# Statsmodel
import statsmodels.tsa.api as smt
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
# Scikit-learn
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error


%matplotlib inline
sns.set()

In [ ]:
# Import the data
df = pd.read_csv("./train.csv", parse_dates=["datetime"])
df.head()

In [ ]:
# Let's get the most important statistics
df.describe()

In [ ]:
# Let's graph the series during the last month. The train data only includes the first 19 days of the month
plt.figure(figsize=(10, 5))
plt.plot(df["count"].values[-456:])
plt.title("Amount of rented bike during the last month")
plt.ylabel("Rented Bikes")
plt.xlabel('Hours')
plt.show()


In [ ]:
# Using some built-in functions, let's determine if the data is seasonal or not. 
# If the autocorrelation plot shows a sinusoidal shape, the data is seasonal (i.e., periodic)
# We prefer seasonal data because it is easier to make predictions on it (using historical data). 
# Lags are previous timesteps

plot_acf(df['count'].values)
plt.xlabel("Lag")
plt.ylabel("Correlation")
plt.show()

In [ ]:
# Another way to have more information is using both the autocorrelation and the partial autocorrelation. 
# Additionally we can perform an hypothesis test to check if our data is stationary or not. 
# The lags are the number of previous time steps

def tsplot(y, lags=None, figsize=(12, 7), syle='bmh'):
    
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
        
    with plt.style.context(style='bmh'):
        # Create figure and axes
        fig = plt.figure(figsize=figsize)
        layout = (2,2)
        ts_ax = plt.subplot2grid(layout, (0,0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1,0))
        pacf_ax = plt.subplot2grid(layout, (1,1))
        
        # Pot the variable
        y.plot(ax=ts_ax)
        # Returns the p_value of the Dickey-Fuller test. If p_value = 1, the process is not stationary
        # If the process is stationary (p_value = 0), then the series has a constant mean.
        p_value = adfuller(y)[1]
        ts_ax.set_title('Time Series Analysis Plots\n Dickey-Fuller: p={0:.5f}'.format(p_value))
        ts_ax.set_xlabel("Samples")
        plot_acf(y, lags=lags, ax=acf_ax)
        acf_ax.set_xlabel("Lag")
        plot_pacf(y, lags=lags, ax=pacf_ax, method="ywm")
        pacf_ax.set_xlabel("Lag")
        plt.tight_layout()
        
tsplot(df["count"].values)

# Notice that the ACF shows a sine-wave shape pattern, the PACF has spikes at lags 1 and 2, 
# and no correlation for other lags, which means a probable AR order of 2 and seasonal AR order of 2
# For the PACF plot, initial spikes at lag = 1 and 2 and seasonal spikes at lag = 24, 
# which means a probable MA order of 2 and seasonal MA order of 2

In [ ]:
# There's another graphical built-in function to see if a process is seasonal or not.
def decompose_bike_sharing_demand(df, share_type='count', samples=250, period=24):
    if samples == 'all':
        res = seasonal_decompose(df[share_type].values, period=period)
    else:
        res = seasonal_decompose(df[share_type].values[-samples:], period=period)
    
    observed = res.observed
    trend = res.trend
    seasonal = res.seasonal
    residual = res.resid

    fig, axs = plt.subplots(4, figsize=(16,16))
    axs[0].set_title('Observed', fontsize=16)
    axs[0].plot(observed)
    axs[0].grid()
    
    axs[1].set_title('Trend', fontsize=16)
    axs[1].plot(trend)
    axs[1].grid()

    axs[2].set_title('Seasonality', fontsize=16)
    axs[2].plot(seasonal)
    axs[2].grid()

    axs[3].set_title('Noise', fontsize=16)
    axs[3].plot(residual)
    axs[3].scatter(y=residual, x=range(len(residual)), alpha=0.5)
    axs[3].grid()
    
    plt.show()

# We will examine the last reported month in periods of 24 hours.
decompose_bike_sharing_demand(df, samples=720, period=24)

## Moving Average

In [ ]:
# Then we need to find a model that allow us to capture the singularities of this process 
# (e.g., mean, variance, seasonality, stationarity) so we are able to predict the future. 

# One of the easiest ways to model time series data is through the Moving Average, 
# which predicts the next value or observation is the mean of past values or observations (called window). 

# The longer the window, the smoother the variations (tendency to the mean), the smaller the window, 
# the most likely the model will follow the trend. 

def plot_moving_average(series, window, plot_intervals=False, scale=1.96):

    rolling_mean = series.rolling(window=window).mean()
    
    plt.figure(figsize=(17,8))
    plt.plot(series[window:], label='Actual values')
    plt.title('Moving average\n window size = {}'.format(window))
    plt.plot(rolling_mean, 'g', label='Rolling mean trend')
    
    #Plot confidence intervals for smoothed values
    if plot_intervals:
        mae = mean_absolute_error(series[window:], rolling_mean[window:])
        deviation = np.std(series[window:] - rolling_mean[window:])
        lower_bound = rolling_mean - (mae + scale * deviation)
        upper_bound = rolling_mean + (mae + scale * deviation)
        plt.plot(upper_bound, 'r--', label='Upper bound / Lower bound')
        plt.plot(lower_bound, 'r--')
            
    
    plt.legend(loc='best')
    plt.grid(True)


In [ ]:
# Smooth by the previous 1 days (24 hours)
plot_moving_average(pd.Series(df["count"].values), window=24, plot_intervals=False)

## SARIMA (Seasonal Autoregressive Integrated Moving Average)

In [ ]:
"""
SARIMA is the combination of the autoregressive part with the moving average. 
This is specially useful to time series that exhibit seasonality.

As its name indicates, the first part is an autoregression (AR). 
Basically, we assume that an observation depends on its previous observations within some lag. 
Therefore, it exists strong correlation between the observations in a period of time. 

In the moving average (MA) part, we assume that there is strong correlation within the window. 

"""

from statsmodels.tsa.statespace.sarimax import SARIMAX


# include I if the data is non stationary
sarima_model = SARIMAX(
    pd.Series(df["count"].values),
    order=(2, 0, 2), # p=order[0], d=order[1], q=order[2], AR(p), I(d), MA(q)
    seasonal_order=(2, 0, 2, 24), # Seasonality: P=seasonal_order[0], D=seasonal_order[1], Q=seasonal_order[2], s=seasonal_order[3]
)
trained_model = sarima_model.fit(disp=-1)
print(trained_model.summary()) 

# Use the trained model to predict 
predictions = trained_model.predict()
# Fix the predictions, we know that we cannot have -1 rented bike
predictions[predictions < 0] =0
print(f"MAPE: {mean_absolute_percentage_error(pd.Series(df['count'].values), predictions)}%")

# Some Results
# SARIMAX(1, 0, 1)x(1, 0, 1, 24) --> MAPE: 0.5182085671253294%
# SARIMAX(0, 1, 0) --> MAPE: 0.5896235403262953%
# SARIMAX(0, 1, 0)x(0, 1, 0, 6)  --> MAPE: 0.8328176193518243%
# SARIMAX(0, 1, 0)x(1, 0, 0, 12) --> MAPE: 0.6259597313147078%
# SARIMAX(2, 0, 0)x(1, 0, 0, 24) --> MAPE: 0.4436969647510891%
# SARIMAX(1, 0, 2)x(1, 0, [1], 24) --> MAPE: 0.5213048599649486%

In [ ]:
# Plot the predictions
plt.figure(figsize=(17,8))
plt.plot(pd.Series(df["count"].values), label='Actual values')
plt.plot(predictions, 'g', label='Predicted values')
plt.legend(loc='best')
plt.grid(True)

## Grid Search of hyperparameters

In [ ]:
#Set initial values and some bounds
ps = range(0, 3)
d = 0  # our data is stationary
qs = range(0, 3)
Ps = range(0, 3)
D = 0  # our data is stationary
Qs = range(0, 3)
s = 24

#Create a list with all possible combinations of parameters
parameters = product(ps, qs, Ps, Qs)
parameters_list = list(parameters)
print(len(parameters_list))


def optimize_SARIMA(parameters_list, d, D, s):
    """
        Return dataframe with parameters and corresponding AIC
        
        parameters_list - list with (p, q, P, Q) tuples
        d - integration order
        D - seasonal integration order
        s - length of season
    """
    
    results = []
    best_aic = float('inf')
    
    for param in tqdm(parameters_list):
        try: model = SARIMAX(pd.Series(df["count"].values), order=(param[0], d, param[1]),
                                               seasonal_order=(param[2], D, param[3], s)).fit(disp=-1)
        except:
            continue
            
        aic = model.aic
        
        #Save best model, AIC and parameters
        if aic < best_aic:
            best_model = model
            best_aic = aic
            best_param = param
        results.append([param, model.aic])
        
    result_table = pd.DataFrame(results)
    result_table.columns = ['parameters', 'aic']
    #Sort in ascending order, lower AIC is better
    result_table = result_table.sort_values(by='aic', ascending=True).reset_index(drop=True)
    
    return result_table

In [ ]:
# Get the results of the hyperparameter optimization
result_table = optimize_SARIMA(parameters_list, d, D, s)

In [ ]:
# At the end the ultimate goal is to predict the last ten days of the total number of rented bikes having the first 20 days. 
# We will do it for the last month but the same can be done for the other months. 

# 19*24=456 --> We have an hourly report
sarima_model = SARIMAX(
    pd.Series(df["count"].values[-456:]),
    order=(2, 0, 2), # p=order[0], d=order[1], q=order[2], AR(p), I(d), MA(q)
    seasonal_order=(2, 0, 2, 24), # Seasonality: P=seasonal_order[0], D=seasonal_order[1], Q=seasonal_order[2], s=seasonal_order[3]
)
# Don't display any warning during training. 
trained_model = sarima_model.fit(disp=-1)
print(trained_model.summary()) 

# We can test the model using the training data, for example, trying to predict the last three days 
predictions = trained_model.predict(start=384, end=455, dynamic=True)
# Fix the predictions, we know that we cannot have -1 rented bike
predictions[predictions < 0] =0
# print(f"MAPE: {mean_absolute_percentage_error(pd.Series(df['count'].values[-456:]), predictions)}%")


# Plot the predictions
plt.figure(figsize=(17,8))
plt.plot(pd.Series(df["count"].values[-456:]), label='Actual values')
plt.plot(predictions, 'g', label='Predicted values')
plt.legend(loc='best')
plt.grid(True)

In [ ]:
# We can use the model to predict the following ten days 
future_predictions = trained_model.predict(start=456, end=695, dynamic=True)
future_predictions.describe()
plt.figure(figsize=(17,8))
plt.plot(future_predictions, label='Forecasted values')
plt.legend(loc='best')
plt.grid(True)

In [ ]:
from pandas.tseries.offsets import DateOffset

# Generate future dates
last_month_df = df.iloc[-456:, :]
last_month_df["datetime"] = pd.to_datetime(last_month_df["datetime"])
last_month_df.set_index("datetime",inplace=True)
last_month_df.index[-1]
future_dates=[last_month_df.index[-1]+ DateOffset(hours=x)for x in range(0,241)]
last_month_df.head()


In [ ]:
# Generate dataframe
future_dates_df=pd.DataFrame(index=future_dates[1:],columns=last_month_df.columns)
future_dates_df['forecast'] = future_predictions.tolist()
future_dates_df.tail()

In [ ]:
# Concatenate the original and newly created dataset
future_df=pd.concat([last_month_df,future_dates_df])
future_df.tail()
# Predict
future_df.head()


In [ ]:
future_df.tail(10)

In [ ]:
# Plot the predictions
plt.figure(figsize=(17,8))
plt.plot(pd.Series(future_df["count"].values), label='Actual values')
plt.plot(pd.Series(future_df["forecast"].values), 'g', label='Predicted values')
plt.legend(loc='best')
plt.grid(True)